# Project 2

In [72]:
import ngsolve
from netgen.geom2d import unit_square
import netgen.gui
from netgen.meshing import *
from ngsolve.webgui import Draw
import numpy as np
from ngsolve import dx

%gui tk

$$
\begin{align*}
(\partial_t \phi, v) + (M \nabla \mu, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu, w) - \left( \gamma (\phi^3 - \phi), w \right) - \left( \epsilon \nabla \phi, \nabla w \right) &= 0 \quad \text{for all } w \in V.
\end{align*}

$$

In [73]:
# Defining constants

epsilon = 0.02
gamma = 50
M = 2

h_max = 0.05
tau = 0.1

In [74]:
# Setting up the mesh

ngmesh = Mesh(dim=2)

N= int(1/h_max)

"""
Code for the square mesh taken from the documentation page
https://docu.ngsolve.org/latest/i-tutorials/unit-4.3-manualmesh/manualmeshing.html
"""


pnums = []
for i in range(N + 1):
    for j in range(N + 1):
        pnums.append(ngmesh.Add(MeshPoint(Pnt(i / N, j / N, 0))))

idx_dom = ngmesh.AddRegion("mat", dim=2)
for j in range(N):
    for i in range(N):
        ngmesh.Add(Element2D(idx_dom, [pnums[i + j * (N + 1)],
                               pnums[i + (j + 1) * (N + 1)],
                               pnums[i + 1 + (j + 1) * (N + 1)],
                               pnums[i + 1 + j * (N + 1)]]))
        
# horizontal boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[N + i * (N + 1)],
                       pnums[N + (i + 1) * (N + 1)]], index=1))
   ngmesh.Add(Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))

# vertical boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[i], pnums[i + 1]], index=2))
   ngmesh.Add(Element1D([pnums[i + N * (N + 1)], pnums[i + 1 + N * (N + 1)]], index=2))


In [83]:
mesh = ngsolve.Mesh(ngmesh)

In [ ]:
Draw(gfu)

Fully explicit:
$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - (\epsilon \nabla \phi^n, \nabla w) &= \left( \gamma ((\phi^n)^3 - \phi^n), w \right) \quad \text{for all } w \in V.
\end{align*}

$$

which can be reformulated as

$$
\begin{align*}
\left( \frac{\phi^{n+1} }{\tau}, v \right) &=  \left( \frac{\phi^n}{\tau}, v \right) - (M \nabla \mu^{n+1}, \nabla v)  \quad \text{for all } v \in V, \\
(\mu^{n+1}, w)  &= (\epsilon \nabla \phi^n, \nabla w) + \left( \gamma ((\phi^n)^3 - \phi^n), w \right) \quad \text{for all } w \in V.
\end{align*}
$$

In [ ]:
# Fully Explicit Scheme

# Defining finite element space

V_phi = ngsolve.H1(mesh,order=1)
V_mu  = ngsolve.H1(mesh,order=1)

phi, v = V_phi.TnT()
mu, w = V_mu.TnT()

gfu = ngsolve.GridFunction(V_phi)


# Initialisation of phi_0

phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
rand_expr = ngsolve.GridFunction(V_phi)
for i in range(mesh.nv):
    rand_expr.vec[i] = phi_0[i]

gfu.Set(rand_expr) 

# Bilinear forms
a_phi = ngsolve.BilinearForm(V_phi)
a_phi += (1/tau)*phi*v*dx

a_mu = ngsolve.BilinearForm(V_mu)
a_mu += mu*w*dx

a_phi.Assemble()
a_mu.Assemble()



# Since it's fully explicit we get linear forms on the rhs (mu is updated before phi)



